In [1]:
import pandas as pd
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()
import os
import openai

In [2]:
# Get the openai api keys
# Get the OpenAI API key and org key from the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate)

human_message_prompt = HumanMessagePromptTemplate(prompt = PromptTemplate(input_variables=["liquor", "cocktail_type", "cuisine", "theme", "spirits_and_amounts"],\
                         template = "Given the following parameters: the name of the liquor {liquor} I am trying to use up, the {spirits_and_amounts} that I have in my inventory,\
                              the type of cocktail {cocktail_type}, the type of cuisine {cuisine} I am looking to pair it with, and the theme {theme},\
                                please help me come up with a creative cocktail with a fun and creative name that doesn't\
                                necessarily include the name of the spirit or the theme.  If you can use the spirits in my inventory in spirits_and_amounts that would be great,\
                                      but if not feel free to use any spirits you think would make a good cocktail with my given parameters  Please be as specific as possible with your instructions.\
                                    \n\nRecipe Name: \n\nIngredients: \n\nInstructions: \n\n")
)

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(model_name = "gpt-3.5-turbo", max_tokens = 1500, temperature = 0.7, top_p = 0.9, frequency_penalty = 0.5, presence_penalty = 0.5,\
            openai_api_key = openai.api_key, openai_organization =  openai.organization)

In [4]:
from langchain.chains import LLMChain
chain = LLMChain(llm = chat, prompt = chat_prompt_template)


In [5]:
from langchain.agents import create_csv_agent


In [6]:
spirits_list = [
    'Absinthe',
    'Agave Nectar',
    'Akvavit',
    'Almond Liqueur',
    'Amaretto',
    'Aperol',
    'Apple Brandy',
    'Applejack',
    'Apricot Brandy',
    'Averna',
    'Bailey\'s Irish Cream',
    'Bénédictine',
    'Bitters',
    'Black Sambuca',
    'Blue Curaçao',
    'Bourbon',
    'Brandy',
    'Campari',
    'Chambord',
    'Cherry Brandy',
    'Cider',
    'Cinnamon Schnapps',
    'Cocchi Americano',
    'Cognac',
    'Cointreau',
    'Crème de Cacao',
    'Crème de Cassis',
    'Crème de Menthe',
    'Crème de Violette',
    'Curacao',
    'Drambuie',
    'Dry Vermouth',
    'Elderflower Liqueur',
    'Falernum',
    'Fernet Branca',
    'Galliano',
    'Genever',
    'Gin',
    'Gold Rum',
    'Goldschläger',
    'Grand Marnier',
    'Grapefruit Liqueur',
    'Green Chartreuse',
    'Grenadine',
    'Hazelnut Liqueur',
    'Hpnotiq',
    'Irish Cream',
    'Jägermeister',
    'Kahlua',
    'Lemon Juice',
    'Lemoncello',
    'Lillet',
    'Lime Juice',
    'Maraschino Liqueur',
    'Mezcal',
    'Midori',
    'Mint Leaves',
    'Nutmeg',
    'Orange Bitters',
    'Orange Curacao',
    'Orange Flower Water',
    'Orange Juice',
    'Orgeat Syrup',
    'Overproof Rum',
    'Peach Bitters',
    'Peach Liqueur',
    'Peppermint Schnapps',
    'Pernod',
    'Peychaud\'s Bitters',
    'Pimm\'s No. 1',
    'Pineapple Juice',
    'Port',
    'Prosecco',
    'Pumpkin Liqueur',
    'Raspberry Liqueur',
    'Red Wine',
    'Rhum Agricole',
    'Rose Water',
    'Rum',
    'Rye Whiskey',
    'Sake',
    'Sambuca',
    'Scotch',
    'Sherry',
    'Simple Syrup',
    'Sloe Gin',
    'Smoked Paprika',
    'Southern Comfort',
    'Sparkling Wine',
    'Spiced Rum',
    'St. Germain',
    'Strawberry Liqueur',
    'Sweet and Sour Mix',
    'Sweet Vermouth',
    'Tabasco Sauce',
    'Tequila',
    'Tomato Juice',
    'Triple Sec',
    'Vanilla Liqueur',
    'Vanilla Syrup',
    'Vermouth',
    'Vodka',
    'Whiskey',
    'White Rum',
    'White Wine',
    'Yellow Chartreuse'
]



In [7]:
# Create a pandas dataframe with the list of spirits
df = pd.DataFrame(spirits_list, columns = ["liquor"])

In [8]:
import numpy as np
# Add a column to the dataframe called "amount" and generate a random number between 1 and 1000 for each row
df["amount"] = np.random.randint(1, 1000, df.shape[0])


In [9]:
# Change the name of the "amount" columns to "amount_in_oz"
df.rename(columns = {"amount": "amount_in_oz"}, inplace = True)
df

,liquor,amount_in_oz
0,Absinthe,537
1,Agave Nectar,941
2,Akvavit,940
3,Almond Liqueur,653
4,Amaretto,741
...,...,...
101,Vodka,660
102,Whiskey,732
103,White Rum,730
104,White Wine,280


In [10]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", max_tokens = 750, temperature = 0.7, top_p = 0.9, frequency_penalty = 0.5, presence_penalty = 0.5,\
            openai_api_key = openai.api_key, openai_organization =  openai.organization)


In [11]:
# Export the dataframe to a csv file
df.to_csv("liquor_amounts.csv", index = False)

In [12]:
# Create the agent using the csv file
agent = create_csv_agent(llm, "liquor_amounts.csv", verbose = True)


In [13]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [19]:
class SpiritsList(BaseModel):
    spirits: List[str] = Field(description = "A list of spirits")
    amounts: List[int] = Field(description = "A list of amounts in oz")

    
spirits_query = "Return a list of spirits in the .csv file and their amounts in oz"

parser = PydanticOutputParser(pydantic_object=SpiritsList, query=spirits_query)

spirits_and_amounts = agent.run(input = "Return a list of spirits and their amounts in oz", parser=parser)


     



> Entering new AgentExecutor chain...
Thought: We need to select only the rows that contain spirits and their respective amounts.
Action: python_repl_ast
Action Input:
spirits_df = df[df['liquor'].str.contains('Liqueur') | df['liquor'].str.contains('Whiskey') | df['liquor'].str.contains('Vodka') | df['liquor'].str.contains('Gin') | df['liquor'].str.contains('Rum')]
spirits_list = spirits_df.values.tolist()
print(spirits_list)[['Almond Liqueur', 653], ['Elderflower Liqueur', 647], ['Gin', 518], ['Gold Rum', 982], ['Grapefruit Liqueur', 309], ['Hazelnut Liqueur', 857], ['Maraschino Liqueur', 608], ['Overproof Rum', 299], ['Peach Liqueur', 268], ['Pumpkin Liqueur', 819], ['Raspberry Liqueur', 948], ['Rum', 309], ['Rye Whiskey', 937], ['Sloe Gin', 86], ['Spiced Rum', 3], ['Strawberry Liqueur', 87], ['Vanilla Liqueur', 341], ['Vodka', 660], ['Whiskey', 732], ['White Rum', 730]]

Observation: None
Thought:We need to modify the input to include only the liquor and amount columns
Action: pyt

In [15]:
spirits_and_amounts

'[(\'Absinthe\', 537), (\'Agave Nectar\', 941), (\'Akvavit\', 940), (\'Almond Liqueur\', 653), (\'Amaretto\', 741), (\'Aperol\', 751), (\'Apple Brandy\', 872), (\'Applejack\', 903), (\'Apricot Brandy\', 164), (\'Averna\', 372), ("Bailey\'s Irish Cream", 913...)]'

In [20]:
print(chain.run(liquor = "ouzo", cocktail_type = "craft", cuisine = "Greek", theme = "summer", spirits_and_amounts = spirits_and_amounts))

Recipe Name: Ouzo Sunset

Ingredients:
- 1 oz ouzo
- 1 oz pineapple juice
- 1/2 oz orange juice
- 1/2 oz lime juice
- 1/2 oz honey syrup (equal parts honey and water)
- Club soda
- Pineapple wedge for garnish

Instructions:
1. Fill a shaker with ice.
2. Add ouzo, pineapple juice, orange juice, lime juice, and honey syrup to the shaker.
3. Shake well until chilled.
4. Strain into a glass filled with ice.
5. Top with club soda.
6. Garnish with a pineapple wedge.

Note: If you don't have honey syrup on hand, you can make it by combining equal parts honey and hot water and stirring until the honey is dissolved.
